In [30]:
import scanpy as sc

In [31]:
adata = sc.read_h5ad("AX4.h5ad")
adata.obsm["X_uce_umap"]

array([[10.479142 ,  6.536511 ],
       [11.502898 ,  6.567293 ],
       [10.937643 ,  6.809363 ],
       ...,
       [-2.517476 ,  9.455943 ],
       [-1.1911128,  7.9335375],
       [-0.3896108,  8.115399 ]], shape=(48983, 2), dtype=float32)

In [32]:
import pandas as pd

# pick only the columns you want from obs
obs_keep = adata.obs[["time", "cell_type"]].copy()
var_keep = adata.var[["gene_ids", "Gene Symbol"]].copy()

# build the new AnnData with the same X/var and only selected obs
adata2 = sc.AnnData(
    X=adata.X,                 # keeps sparse/dense as-is
    obs=obs_keep,
    var=var_keep
)

# copy just the embedding you care about
adata2.obsm["X_uce_umap"] = adata.obsm["X_uce_umap"]


In [33]:
adata2

AnnData object with n_obs × n_vars = 48983 × 12883
    obs: 'time', 'cell_type'
    var: 'gene_ids', 'Gene Symbol'
    obsm: 'X_uce_umap'

In [24]:
for k in ("connectivities", "distances"):
    if k in adata2.obsp: del adata2.obsp[k]
if "neighbors" in adata2.uns: del adata2.uns["neighbors"]
for k in ("rank_genes_groups", "dendrogram", "paga", "leiden", "louvain"):
    if k in adata2.uns: del adata2.uns[k]


In [26]:
import scipy.sparse as sp
if not sp.issparse(adata.X):
    adata.X = sp.csr_matrix(adata.X) 
adata.X = adata.X.astype("uint32")     # or "uint16" if max <= 65535



In [34]:
adata2.write("AX4.h5ad.gz", compression="gzip")

: 